In [ ]:
print('Setup complete.')

In [ ]:
# Install dependencies - AskSage and other required packages
!pip install asksageclient
!pip install pytest
!pip install pandas
!pip install matplotlib
!pip install requests
!pip install python-dotenv

print("✅ Dependencies installed successfully!")

# Import all required libraries
import subprocess
import os
import re
from typing import List, Dict, Tuple, Optional
import json
from pathlib import Path
import datetime
from asksageclient import AskSageClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("📦 All imports successful!")

# Safe Command-Line Agent Demo with AskSage gpt-5-mini

This notebook demonstrates how to build a safe command-line agent that can execute shell commands with strict safety controls, enhanced with AI reasoning capabilities using AskSage's gpt-5-mini model.

**Learning Objectives:**
- Understand safe shell command execution with AI oversight
- Implement command whitelisting and validation with AI assistance
- Build an intelligent agent that can locate files, analyze findings, and provide AI-powered insights
- Practice security-first agent design with AI enhancement

**Allowed Commands:** `ls`, `cat`, `grep -n`, `find`

**AI Model:** gpt-5-mini via AskSage API

**Note:** This demo requires AskSage API credentials. Set your API_KEY and EMAIL as environment variables.

## 1. AskSage Client Setup

First, let's set up the AskSage client to use gpt-5-mini for AI-powered analysis.

In [ ]:
# Setup AskSage client with gpt-5-mini
class AskSageManager:
    """Manager for AskSage API interactions using gpt-5-mini."""
    
    def __init__(self):
        # Get credentials from environment variables
        self.email = os.getenv('ASKSAGE_EMAIL')
        self.api_key = os.getenv('ASKSAGE_API_KEY')
        
        if not self.email or not self.api_key:
            raise ValueError("Please set ASKSAGE_EMAIL and ASKSAGE_API_KEY environment variables")
        
        # Initialize AskSage client
        self.client = AskSageClient(self.email, self.api_key)
        self.model = "gpt-5-mini"  # Using gpt-5-mini as requested
        
        print(f"🤖 AskSage client initialized with {self.model}")
        
    def analyze_command_safety(self, command: str) -> Dict:
        """Use AI to analyze command safety beyond basic pattern matching."""
        prompt = f"""
        Analyze the safety of this shell command: `{command}`
        
        Consider:
        1. Is this command safe to execute in a read-only exploration context?
        2. Are there any potential security risks or destructive operations?
        3. Does it attempt file modification, deletion, or system changes?
        4. Is it trying to execute dangerous patterns like command injection?
        
        Respond in JSON format:
        {{
            "safe": true/false,
            "risk_level": "low"/"medium"/"high",
            "explanation": "Brief explanation of the assessment",
            "suggestions": "Any safer alternatives if applicable"
        }}
        """
        
        try:
            response = self.client.query(prompt, model=self.model)
            # Parse AI response
            ai_analysis = json.loads(response.get('response', '{}'))
            return ai_analysis
        except Exception as e:
            print(f"⚠️ AI analysis failed: {e}")
            return {"safe": False, "risk_level": "high", "explanation": "AI analysis unavailable", "suggestions": "Manual review required"}
    
    def analyze_findings(self, findings_data: Dict) -> str:
        """Use AI to provide intelligent analysis of command execution findings."""
        prompt = f"""
        Analyze these command execution findings and provide insights:
        
        {json.dumps(findings_data, indent=2)}
        
        Please provide:
        1. A summary of what was discovered
        2. Any interesting patterns or notable findings
        3. Recommendations for further exploration
        4. Potential security or code quality observations
        
        Keep the response concise but informative.
        """
        
        try:
            response = self.client.query(prompt, model=self.model)
            return response.get('response', 'AI analysis unavailable')
        except Exception as e:
            return f"AI analysis failed: {e}"

# Initialize the AskSage manager
try:
    ai_manager = AskSageManager()
    print("🎯 AskSage gpt-5-mini ready for intelligent command analysis!")
except Exception as e:
    print(f"❌ Failed to initialize AskSage: {e}")
    print("Please ensure ASKSAGE_EMAIL and ASKSAGE_API_KEY environment variables are set")
    ai_manager = None

## 2. Enhanced Safe Command Executor with AI

Now let's build an enhanced safe command executor that uses both traditional validation and AI-powered safety analysis.

In [ ]:
class AIEnhancedSafeCommandExecutor:
    """A safe command executor enhanced with AI-powered safety analysis."""
    
    def __init__(self, working_directory: str = ".", ai_manager=None):
        self.working_directory = os.path.abspath(working_directory)
        self.command_log = []
        self.ai_manager = ai_manager
        
        # Whitelist of allowed commands with their patterns
        self.allowed_commands = {
            'ls': r'^ls(\s+-[alhtrS]*)?\s*([\w\./\-]*)?$',
            'cat': r'^cat\s+([\w\./\-]+)$',
            'grep': r'^grep\s+-n\s+["\']?([^"\';|&<>]+)["\']?\s+([\w\./\-]+)$',
            'find': r'^find\s+([\w\./\-]*)\s+-name\s+["\']?([^"\';|&<>]+)["\']?$'
        }
    
    def validate_command(self, command: str) -> Tuple[bool, str, Dict]:
        """Enhanced validation using both pattern matching and AI analysis."""
        command = command.strip()
        ai_analysis = {}
        
        # Traditional pattern-based validation
        dangerous_patterns = [';', '|', '&', '>', '<', '$(', '`', 'rm', 'mv', 'cp', 'chmod', 'sudo']
        for pattern in dangerous_patterns:
            if pattern in command:
                return False, f"Dangerous pattern detected: {pattern}", ai_analysis
        
        # Check whitelist
        whitelisted = False
        for cmd_name, pattern in self.allowed_commands.items():
            if re.match(pattern, command):
                whitelisted = True
                break
        
        if not whitelisted:
            return False, "Command not in whitelist", ai_analysis
        
        # AI-enhanced validation if available
        if self.ai_manager:
            print(f"🔍 Running AI safety analysis on: {command}")
            ai_analysis = self.ai_manager.analyze_command_safety(command)
            
            if not ai_analysis.get('safe', True):
                return False, f"AI flagged as unsafe: {ai_analysis.get('explanation', 'Unknown risk')}", ai_analysis
            
            print(f"✅ AI analysis: {ai_analysis.get('explanation', 'Safe to execute')}")
        
        return True, "Command validated (traditional + AI)", ai_analysis
    
    def execute_command(self, command: str) -> Dict:
        """Execute a validated command safely with AI insights."""
        is_valid, validation_msg, ai_analysis = self.validate_command(command)
        
        result = {
            'command': command,
            'valid': is_valid,
            'validation_msg': validation_msg,
            'ai_analysis': ai_analysis,
            'output': '',
            'error': '',
            'success': False
        }
        
        if not is_valid:
            result['error'] = f"Command rejected: {validation_msg}"
            self.command_log.append(result)
            return result
        
        try:
            # Execute command in safe environment
            process = subprocess.run(
                command,
                shell=True,
                cwd=self.working_directory,
                capture_output=True,
                text=True,
                timeout=30  # 30 second timeout
            )
            
            result['output'] = process.stdout
            result['error'] = process.stderr
            result['success'] = process.returncode == 0
            
        except subprocess.TimeoutExpired:
            result['error'] = "Command timed out"
        except Exception as e:
            result['error'] = f"Execution error: {str(e)}"
        
        self.command_log.append(result)
        return result
    
    def get_command_log(self) -> List[Dict]:
        """Get the full command execution log."""
        return self.command_log.copy()

print("🔐 AI-Enhanced Safe Command Executor implemented!")

## 3. Intelligent Command-Line Agent

Now let's build an intelligent agent that can explore filesystems and provide AI-powered insights using gpt-5-mini.

In [ ]:
class IntelligentCommandAgent:
    """An intelligent agent that safely explores filesystems with AI-powered analysis."""
    
    def __init__(self, working_directory: str = ".", ai_manager=None):
        self.executor = AIEnhancedSafeCommandExecutor(working_directory, ai_manager)
        self.ai_manager = ai_manager
        self.findings = []
    
    def explore_directory(self, path: str = ".") -> str:
        """List contents of a directory with AI insights."""
        print(f"🔍 Exploring directory: {path}")
        result = self.executor.execute_command(f"ls -la {path}")
        
        if result['success']:
            print(result['output'])
            return result['output']
        else:
            print(f"❌ Error: {result['error']}")
            return ""
    
    def find_files_by_name(self, pattern: str, search_path: str = ".") -> List[str]:
        """Find files matching a name pattern."""
        print(f"🔍 Finding files matching '{pattern}' in {search_path}")
        result = self.executor.execute_command(f"find {search_path} -name '{pattern}'")
        
        if result['success']:
            files = [f.strip() for f in result['output'].split('\n') if f.strip()]
            print(f"📁 Found {len(files)} files:")
            for file in files[:10]:  # Limit output
                print(f"  - {file}")
            if len(files) > 10:
                print(f"  ... and {len(files) - 10} more")
            return files
        else:
            print(f"❌ Error: {result['error']}")
            return []
    
    def search_in_file(self, filepath: str, pattern: str) -> List[str]:
        """Search for a pattern in a specific file."""
        print(f"🔍 Searching for '{pattern}' in {filepath}")
        result = self.executor.execute_command(f"grep -n '{pattern}' {filepath}")
        
        if result['success']:
            matches = [m.strip() for m in result['output'].split('\n') if m.strip()]
            print(f"📝 Found {len(matches)} matches:")
            for match in matches[:5]:  # Limit output
                print(f"  {match}")
            if len(matches) > 5:
                print(f"  ... and {len(matches) - 5} more")
            return matches
        else:
            print(f"❌ No matches or error: {result['error']}")
            return []
    
    def read_file_preview(self, filepath: str, lines: int = 20) -> str:
        """Read the first few lines of a file."""
        print(f"📖 Reading preview of {filepath}")
        result = self.executor.execute_command(f"cat {filepath}")
        
        if result['success']:
            content_lines = result['output'].split('\n')
            preview = '\n'.join(content_lines[:lines])
            if len(content_lines) > lines:
                preview += f"\n... (showing first {lines} of {len(content_lines)} lines)"
            print(preview)
            return result['output']
        else:
            print(f"❌ Error reading file: {result['error']}")
            return ""
    
    def intelligent_symbol_search(self, symbol: str, search_path: str = ".") -> Dict:
        """Locate a symbol with AI-powered analysis of findings."""
        print(f"\n🎯 === INTELLIGENT SYMBOL SEARCH: '{symbol}' (powered by gpt-5-mini) ===")
        
        findings = {
            'symbol': symbol,
            'search_path': search_path,
            'files_found': [],
            'matches': [],
            'ai_insights': '',
            'summary': ''
        }
        
        # Step 1: Find Python files that might contain the symbol
        print("\n📂 Step 1: Finding Python files...")
        py_files = self.find_files_by_name("*.py", search_path)
        findings['files_found'] = py_files
        
        # Step 2: Search for the symbol in each Python file
        print(f"\n🔎 Step 2: Searching for '{symbol}' in {len(py_files)} Python files...")
        total_matches = 0
        
        for py_file in py_files[:20]:  # Limit to first 20 files
            matches = self.search_in_file(py_file, symbol)
            if matches:
                findings['matches'].append({
                    'file': py_file,
                    'matches': matches
                })
                total_matches += len(matches)
        
        # Step 3: AI-powered analysis of findings
        print(f"\n🧠 Step 3: AI Analysis with gpt-5-mini...")
        if self.ai_manager and findings['matches']:
            findings['ai_insights'] = self.ai_manager.analyze_findings(findings)
            print(f"\n🤖 AI Insights:\n{findings['ai_insights']}")
        
        # Step 4: Generate summary
        print(f"\n📊 Step 4: Generating summary...")
        summary_parts = [
            f"Symbol '{symbol}' search results:",
            f"- Searched in: {search_path}",
            f"- Python files found: {len(py_files)}",
            f"- Files with matches: {len(findings['matches'])}",
            f"- Total matches: {total_matches}"
        ]
        
        if findings['matches']:
            summary_parts.append("\nFiles containing the symbol:")
            for match_info in findings['matches'][:5]:  # Top 5 files
                summary_parts.append(f"  - {match_info['file']} ({len(match_info['matches'])} matches)")
        
        findings['summary'] = '\n'.join(summary_parts)
        print(f"\n📋 SUMMARY:\n{findings['summary']}")
        
        self.findings.append(findings)
        return findings
    
    def generate_intelligent_report(self) -> str:
        """Generate a comprehensive report enhanced with AI insights."""
        report_parts = [
            "# Intelligent Command-Line Agent Report",
            f"Generated at: {datetime.datetime.now()}",
            f"AI Model: gpt-5-mini via AskSage",
            "\n## Command Execution Log"
        ]
        
        # Add command log with AI analysis
        for i, cmd in enumerate(self.executor.get_command_log(), 1):
            status = "✅" if cmd['success'] else "❌"
            report_parts.append(f"{i}. {status} `{cmd['command']}`")
            
            if cmd.get('ai_analysis'):
                ai_info = cmd['ai_analysis']
                report_parts.append(f"   🤖 AI Risk Level: {ai_info.get('risk_level', 'unknown')}")
                report_parts.append(f"   🤖 AI Analysis: {ai_info.get('explanation', 'N/A')}")
            
            if not cmd['success']:
                report_parts.append(f"   Error: {cmd['error']}")
        
        # Add findings summary with AI insights
        if self.findings:
            report_parts.extend(["\n## Intelligent Search Results", ""])
            for finding in self.findings:
                report_parts.append(f"### Symbol: {finding['symbol']}")
                report_parts.append(finding['summary'])
                if finding.get('ai_insights'):
                    report_parts.append("\n#### AI Insights:")
                    report_parts.append(finding['ai_insights'])
                report_parts.append("")
        
        return "\n".join(report_parts)

print("🤖 Intelligent Command Agent with gpt-5-mini implemented!")

## 4. Demo: Intelligent Agent in Action!

Now let's demonstrate our AI-enhanced safe command-line agent using gpt-5-mini for intelligent analysis.

In [ ]:
# Initialize the intelligent agent
import os
current_dir = os.getcwd()
agent = IntelligentCommandAgent(current_dir, ai_manager)

print(f"🤖 Intelligent Command-Line Agent initialized in: {current_dir}")
print("🔒 Security: Only ls, cat, grep -n, and find commands allowed")
print("🧠 AI Enhancement: gpt-5-mini via AskSage for intelligent analysis")

In [ ]:
# Demo 1: Explore the current directory structure with AI insights
print("=== DEMO 1: Exploring Current Directory Structure (AI-Enhanced) ===")
directory_content = agent.explore_directory(".")

# Get AI insights about the directory structure if available
if ai_manager and directory_content:
    print("\n🧠 Getting AI insights about directory structure...")
    insights_prompt = f"Analyze this directory listing and provide insights about the project structure:\n\n{directory_content[:1000]}"  # Limit length
    try:
        response = ai_manager.client.query(insights_prompt, model=ai_manager.model)
        print(f"\n🤖 AI Insights: {response.get('response', 'Analysis unavailable')}")
    except Exception as e:
        print(f"AI insights unavailable: {e}")

In [ ]:
# Demo 2: Find all Python files with AI-powered analysis
print("\n=== DEMO 2: Finding Python Files with AI Analysis ===")
py_files = agent.find_files_by_name("*.py", ".")

# If no Python files, try other file types
if not py_files:
    print("\nNo .py files found, looking for .ipynb files...")
    notebook_files = agent.find_files_by_name("*.ipynb", ".")
    if notebook_files and ai_manager:
        print("\n🧠 AI analyzing notebook files...")
        try:
            analysis_prompt = f"These notebook files were found: {notebook_files[:5]}. What type of project might this be?"
            response = ai_manager.client.query(analysis_prompt, model=ai_manager.model)
            print(f"🤖 AI Project Analysis: {response.get('response', 'Analysis unavailable')}")
        except Exception as e:
            print(f"AI analysis error: {e}")

In [ ]:
# Demo 3: Intelligent symbol search with gpt-5-mini analysis
print("\n=== DEMO 3: Intelligent Symbol Search - 'import' (gpt-5-mini Enhanced) ===")
import_findings = agent.intelligent_symbol_search("import", ".")

# If no Python files with imports, try searching for function definitions
if not import_findings['matches']:
    print("\nNo Python imports found, searching for 'def' functions...")
    def_findings = agent.intelligent_symbol_search("def", ".")

In [ ]:
# Demo 4: Read file preview with AI-powered content analysis
print("\n=== DEMO 4: Reading File Preview with AI Analysis ===")

# Try to read the first available file from our search
file_content = None
target_file = None

if import_findings.get('matches'):
    target_file = import_findings['matches'][0]['file']
    file_content = agent.read_file_preview(target_file, 15)
else:
    # Try to find and read any text file
    text_files = agent.find_files_by_name("*.txt", ".")
    if text_files:
        target_file = text_files[0]
        file_content = agent.read_file_preview(target_file, 15)
    else:
        print("📝 No suitable files found for preview. Creating a sample file...")
        sample_code = "# Sample Python code\nimport os\nimport sys\n\ndef hello_world():\n    print('Hello, World!')\n\nif __name__ == '__main__':\n    hello_world()\n"
        with open("sample_code.py", "w") as f:
            f.write(sample_code)
        target_file = "sample_code.py"
        file_content = agent.read_file_preview(target_file, 10)

# AI-powered content analysis
if ai_manager and file_content and target_file:
    print(f"\n🧠 AI analyzing content of {target_file}...")
    try:
        content_prompt = f"Analyze this code/file content and provide insights about its purpose, quality, and any notable patterns:\n\n{file_content[:800]}"  # Limit content length
        response = ai_manager.client.query(content_prompt, model=ai_manager.model)
        print(f"\n🤖 AI Content Analysis: {response.get('response', 'Analysis unavailable')}")
    except Exception as e:
        print(f"AI content analysis error: {e}")

## 5. Enhanced Security Demonstration with AI

Let's test our security measures, now enhanced with AI-powered command analysis.

In [ ]:
# Test dangerous commands - these should be rejected by both traditional and AI analysis
dangerous_commands = [
    "rm -rf /",
    "ls; cat /etc/passwd",
    "find . -name '*.py' | xargs rm",
    "cat file.txt > dangerous.txt",
    "grep pattern file.txt && echo 'hacked'",
    "curl -X POST malicious-site.com",  # New: network command
    "python -c 'import os; os.system(\"rm -rf /\")'",  # New: code injection
]

print("🛡️ === ENHANCED SECURITY TEST: Traditional + AI Analysis ===")
print("Testing dangerous commands against both pattern matching and gpt-5-mini analysis...\n")

for i, cmd in enumerate(dangerous_commands, 1):
    print(f"{i}. Testing: {cmd}")
    result = agent.executor.execute_command(cmd)
    
    if result['valid']:
        print("   ⚠️  WARNING: Command was allowed!")
    else:
        print(f"   ✅ Blocked: {result['validation_msg']}")
        
        # Show AI analysis if available
        if result.get('ai_analysis'):
            ai_info = result['ai_analysis']
            print(f"   🤖 AI Risk Assessment: {ai_info.get('risk_level', 'unknown').upper()}")
            print(f"   🤖 AI Reasoning: {ai_info.get('explanation', 'N/A')}")
    print()

print("🎉 All dangerous commands successfully blocked by enhanced security!")

## 6. Generate Comprehensive AI-Enhanced Report

Let's generate a final report that includes both traditional command logging and AI-powered insights.

In [ ]:
# Generate and display the comprehensive AI-enhanced report
print("📊 === GENERATING AI-ENHANCED FINAL REPORT ===")
report = agent.generate_intelligent_report()
print(report)

# Get overall session insights from AI
if ai_manager:
    print("\n\n🧠 === AI SESSION SUMMARY (gpt-5-mini) ===")
    try:
        session_prompt = f"""
        Provide a comprehensive summary of this command-line agent session. 
        
        Key activities performed:
        - Directory exploration
        - File discovery and analysis
        - Symbol/pattern searching
        - Security testing with dangerous commands
        - AI-enhanced safety analysis
        
        Number of commands executed: {len(agent.executor.get_command_log())}
        Number of intelligent searches: {len(agent.findings)}
        
        Provide insights about:
        1. Overall session effectiveness
        2. Security posture observed
        3. Recommendations for future exploration
        4. Notable patterns or discoveries
        """
        
        response = ai_manager.client.query(session_prompt, model=ai_manager.model)
        print(f"\n🤖 AI Session Summary:\n{response.get('response', 'Summary unavailable')}")
    except Exception as e:
        print(f"AI session summary error: {e}")

In [ ]:
# Save the enhanced report to a file
report_filename = "ai_enhanced_command_log_report.md"
with open(report_filename, "w") as f:
    f.write(report)

print(f"💾 AI-Enhanced Report saved to '{report_filename}'")

# In Google Colab, offer download option
try:
    from google.colab import files
    print("📥 Download report using the files panel or run: files.download('ai_enhanced_command_log_report.md')")
except ImportError:
    print("📁 Report available in current directory")

print("\n🎉 AI-Enhanced Demo completed successfully!")
print("\n📝 Key Takeaways:")
print("   1. ✅ Traditional whitelist + pattern matching provides baseline security")
print("   2. 🤖 AI analysis with gpt-5-mini adds intelligent safety assessment")
print("   3. 🧠 AI provides contextual insights about discovered files and patterns")
print("   4. 📊 Enhanced logging captures both traditional and AI-powered analysis")
print("   5. 🔒 Multi-layered security prevents dangerous operations effectively")
print("   6. 🌐 Cross-platform compatibility (Colab + local environments)")
print("   7. 🎯 AskSage integration demonstrates practical AI-enhanced tooling")

# Display model information
if ai_manager:
    print(f"\n🔧 AI Model Used: {ai_manager.model}")
    print(f"📧 AskSage Account: {ai_manager.email}")
else:
    print("\n⚠️  AI features were disabled due to missing credentials")
    print("   Set ASKSAGE_EMAIL and ASKSAGE_API_KEY environment variables to enable AI analysis")